## Seqence to Sequence Model
- 하나의 텍스트 문장이 입력으로 들어오면 하나의 텍스트 문장을 출력하는 구조
- Encoder -> (Context Vector) -> Decoder
    - 입력된 데이터를 Encoder를 거쳐 Context Vector라는 하나의 함축된 데이터로 변경(Encoder 마지막의 은닉계층에서 압축과정 중 손실 발생 -> Attention Mechanism)
    - 벡터가 Decoder를 거쳐 Label에 맞게 학습

In [ ]:
%pip install numpy
%pip install pandas
%pip install matplot
%pip install wordcloud

%pip install konlpy

In [ ]:
# Module import
import os
import re
import json

import numpy as np
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt

In [ ]:
# Settings(특수문자 제거, 특수 토큰 정의)
FILTERS = "([~.,!?\"':;)(])"
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER =[PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)

MAX_SEQ = 25

In [ ]:
# Data Load
def load_data(path):
    data_df = pd.read_csv(path, header=0)
    qst, ans = list(data_df['Q']), list(data_df['A'])
    
    return qst, ans

In [ ]:
# 특수문자 제거 및 전체 문장에 대한 단어 사전 리스트로 반환
def data_tokenizer(data):
    # array to append tokenized data
    words = []
    for sentence in data:
        # FILTERS 내의 값들을 정규화 표현식을 통해서 모두 문자로 변환
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
    # tokeninzing, 정규화를 거친 값들을 반환
    return [word for word in words if word]

In [ ]:
# 형태소 기준 txt_tokenizing 후 각 문장을 형태소들의 리스트로 반환
def prepro_like_morpheme(data):
    morph_analyzer = Okt()
    result_data = []
    for seq in tqdm(data):
        morphemic_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
        result_data.append(morphemic_seq)
    return result_data

In [ ]:
# Generate Word Dictionary
# Return the values (word:index), (index:word), (num_of_total_word)
def load_vocabulary(path, vocab_path, tokenize_as_morph=False):
    # List for dictionary
    vocab_list = []
    # Checking the file presence on the path
    if not os.path.exists(vocab_path):
        if (os.path.exists(path)):
            # load data
            data_df = pd.read_csv(path, encoding='utf-8')
            # array for questions and answers
            qst, ans = list(data_df['Q']), list(data_df['A'])
            if tokenize_as_morph:
                # 형태소에 따른 tokeninzing
                qst = prepro_like_morpheme(qst)
                ans = prepro_like_morpheme(ans)
            data = []
            # .extend(전달값) : 전달값(iterable)을 하나씩 분해해서 리스트에 각 원소로 저장(!! .append()와 구분)
            data.extend(qst)
            data.extend(ans)
            
            words = data_tokenizer(data)
            # 중복 단어 제거를 위해 set
            words = list(set(words))
            # MARKER를 사전에 추가
            # 순서대로 넣기 위해서 인덱스 0 에 추가
            # PAD = "<PADDING>"
            # STD = "<START>"
            # END = "<END>"
            # UNK = "<UNKOWN>"
            words[:0] = MARKER
        
        # 리스트로 된 단어 사전을 vocabulary_file에 저장
        # 각 단어는 개행 문자('\n')를 오른쪽에 달고 저장
        with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word + '\n')
    
    # 사전 파일이 존재하면 파일을 불러스 리스트에 추가
    # 각 개행 문자를 .strip()으로 제거 후 사전 리스트로 저장
    with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocab_list.append(line.strip())
    
    # 리스트 내용을 key:value 형태의 dictionary 구조로 생성
    chr2idx, idx2chr = make_vocabulary(vocab_list)
    # 두 가지 형태의 key, value 형태를 리턴
    return chr2idx, idx2chr, len(chr2idx)

In [ ]:
# 단어:인덱스, 인덱스:단어, 단어 개수 사전 반환 함수
def make_vocabulary(vocab_list):
    chr2idx = {chr:idx for idx, chr in enumerate(vocab_list)}
    idx2chr = {idx:chr for idx, chr in enumerate(vocab_list)}
    return chr2idx, idx2chr